# Deep Q-Learning 

For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will train an agent to play the famous game of __Breakout__.

In [1]:
import sys
import gym
import torch
import pylab
import random
import numpy as np
import time
from collections import deque
from datetime import datetime
from copy import deepcopy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from utils import *
from agent import *
from model import *
from config import *
from env import GameEnv
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Understanding the environment

In the following cell, we initialise our game of __Breakout__ and you can see how the environment looks like. For further documentation of the of the environment refer to https://gym.openai.com/envs. 

In [2]:
envs = []
for i in range(num_envs):
    envs.append(GameEnv('SpaceInvadersDeterministic-v4'))
#env.render()

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [3]:
number_lives = envs[0].life
state_size = envs[0].observation_space.shape
action_size = envs[0].action_space.n
rewards, episodes = [], []

## Creating a DQN Agent

Here we create a DQN Agent. This agent is defined in the __agent.py__. The corresponding neural network is defined in the __model.py__. 

__Evaluation Reward__ : The average reward received in the past 100 episodes/games.

__Frame__ : Number of frames processed in total.

__Memory Size__ : The current size of the replay memory.

In [4]:
agent = Agent(action_size)
torch.save(agent.policy_net.state_dict(), "./save_model/spaceinvaders_ppo_best")
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0
reset_max = 10

### Main Training Loop

In [ ]:
vis_env_idx = 0
vis_env = envs[vis_env_idx]
e = 0
frame = 0
max_eval = -np.inf
reset_count = 0

while (frame < 10000000):
    step = 0
    assert(num_envs * env_mem_size == train_frame)
    frame_next_vals = []
    for i in range(num_envs):
        env = envs[i]
        #history = env.history
        #life = env.life
        #state, reward, done, info = [env.state, env.reward, env.done, env.info]
        for j in range(env_mem_size):
            step += 1
            frame += 1
            
            curr_state = env.history[HISTORY_SIZE-1,:,:]
            action, value = agent.get_action(np.float32(env.history[:HISTORY_SIZE,:,:]) / 255.)
            
            next_state, env.reward, env.done, env.info = env.step(action)
            
            if (i == vis_env_idx):
                vis_env._env.render()
            
            frame_next_state = get_frame(next_state)
            env.history[HISTORY_SIZE,:,:] = frame_next_state
            terminal_state = check_live(env.life, env.info['ale.lives'])
            
            env.life = env.info['ale.lives']
            r = env.reward
            
            agent.memory.push(i, deepcopy(curr_state), action, r, terminal_state, value, 0, 0)
            if (j == env_mem_size-1):
                _, frame_next_val = agent.get_action(np.float32(env.history[1:,:,:]) / 255.)
                frame_next_vals.append(frame_next_val)
            env.score += r
            env.history[:HISTORY_SIZE, :, :] = env.history[1:,:,:]
            
            if (env.done):
                if (e % 50 == 0):
                    print('now time : ', datetime.now())
                    rewards.append(np.mean(evaluation_reward))
                    episodes.append(e)
                    pylab.plot(episodes, rewards, 'b')
                    pylab.savefig("./save_graph/spaceinvaders_ppo.png")
                    torch.save(agent.policy_net, "./save_model/spaceinvaders_ppo")
                    
                    if np.mean(evaluation_reward) > max_eval:
                        torch.save(agent.policy_net.state_dict(), "./save_model/spaceinvaders_ppo_best")
                        max_eval = float(np.mean(evaluation_reward))
                        reset_count = 0
                    elif e > 5000:
                        reset_count += 1
                        """
                        if (reset_count == reset_max):
                            print("Training went nowhere, starting again at best model")
                            agent.policy_net.load_state_dict(torch.load("./save_model/spaceinvaders_ppo_best"))
                            agent.update_target_net()
                            reset_count = 0
                        """
                e += 1
                evaluation_reward.append(env.score)
                print("episode:", e, "  score:", env.score,  " epsilon:", agent.epsilon, "   steps:", step,
                  " evaluation reward:", np.mean(evaluation_reward))
                
                env.done = False
                env.score = 0
                env.history = np.zeros([HISTORY_SIZE+1,84,84], dtype=np.uint8)
                env.state = env.reset()
                env.life = number_lives
                get_init_state(env.history, env.state)
                
                
                
    agent.train_policy_net(frame, frame_next_vals)
    agent.update_target_net()

'''

/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/model.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(x[:,:self.action_size] - torch.max(x[:,:self.action_size],1)[0].unsqueeze(1))
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Training network. lr: 0.000250. clip: 0.100000


/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:255: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  pol_loss += pol_avg.detach().cpu()[0]
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:256: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  vf_loss += value_loss.detach().cpu()[0]
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:257: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  ent_total += ent.detach().cpu()[0]


Iteration 1: Policy loss: -1.094032. Value loss: 6.568430. Entropy: 1.787648.
Iteration 2: Policy loss: -1.059619. Value loss: 5.662950. Entropy: 1.787355.
Iteration 3: Policy loss: -1.087144. Value loss: 5.936832. Entropy: 1.787368.
Training network. lr: 0.000250. clip: 0.100000
Iteration 4: Policy loss: -3.762961. Value loss: 23.942623. Entropy: 1.753100.
Iteration 5: Policy loss: -3.626564. Value loss: 21.021532. Entropy: 1.765004.
Iteration 6: Policy loss: -3.676214. Value loss: 18.459532. Entropy: 1.769020.
now time :  2019-02-26 12:27:11.380270
episode: 1   score: 105.0  epsilon: 1.0    steps: 255  evaluation reward: 105.0


/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

Training network. lr: 0.000221. clip: 0.088333
Iteration 3829: Policy loss: 0.305742. Value loss: 16.793684. Entropy: 0.337647.
Iteration 3830: Policy loss: 0.465713. Value loss: 9.843289. Entropy: 0.316702.
Iteration 3831: Policy loss: 0.267997. Value loss: 7.564712. Entropy: 0.322111.
Training network. lr: 0.000221. clip: 0.088333
Iteration 3832: Policy loss: -1.599959. Value loss: 20.155142. Entropy: 0.205276.
Iteration 3833: Policy loss: -1.626586. Value loss: 12.687953. Entropy: 0.229731.
Iteration 3834: Policy loss: -1.524165. Value loss: 11.063786. Entropy: 0.217740.
episode: 1579   score: 205.0  epsilon: 1.0    steps: 247  evaluation reward: 251.65
episode: 1580   score: 195.0  epsilon: 1.0    steps: 620  evaluation reward: 249.95
Training network. lr: 0.000221. clip: 0.088333
Iteration 3835: Policy loss: -1.563531. Value loss: 28.770050. Entropy: 0.270088.
Iteration 3836: Policy loss: -1.218867. Value loss: 18.720926. Entropy: 0.271002.
Iteration 3837: Policy loss: -1.343923. 

Iteration 3891: Policy loss: -0.928824. Value loss: 13.075006. Entropy: 0.199052.
episode: 1605   score: 75.0  epsilon: 1.0    steps: 83  evaluation reward: 213.75
episode: 1606   score: 210.0  epsilon: 1.0    steps: 558  evaluation reward: 209.2
Training network. lr: 0.000220. clip: 0.088176
Iteration 3892: Policy loss: -0.051510. Value loss: 20.957813. Entropy: 0.284149.
Iteration 3893: Policy loss: -0.409697. Value loss: 14.015271. Entropy: 0.264501.
Iteration 3894: Policy loss: -0.217371. Value loss: 11.035592. Entropy: 0.307459.
episode: 1607   score: 215.0  epsilon: 1.0    steps: 355  evaluation reward: 210.45
Training network. lr: 0.000220. clip: 0.088176
Iteration 3895: Policy loss: -1.987573. Value loss: 31.814426. Entropy: 0.288151.
Iteration 3896: Policy loss: -2.272815. Value loss: 22.876772. Entropy: 0.293534.
Iteration 3897: Policy loss: -2.160172. Value loss: 15.561296. Entropy: 0.309155.
episode: 1608   score: 315.0  epsilon: 1.0    steps: 844  evaluation reward: 208.15

Training network. lr: 0.000220. clip: 0.087872
Iteration 3952: Policy loss: -0.887297. Value loss: 40.823978. Entropy: 0.115070.
Iteration 3953: Policy loss: -0.883693. Value loss: 27.594193. Entropy: 0.074280.
Iteration 3954: Policy loss: -0.971265. Value loss: 22.497408. Entropy: 0.090220.
episode: 1633   score: 105.0  epsilon: 1.0    steps: 101  evaluation reward: 208.6
episode: 1634   score: 80.0  epsilon: 1.0    steps: 496  evaluation reward: 206.65
episode: 1635   score: 530.0  epsilon: 1.0    steps: 725  evaluation reward: 209.3
Training network. lr: 0.000220. clip: 0.087872
Iteration 3955: Policy loss: 2.697236. Value loss: 30.571947. Entropy: 0.109397.
Iteration 3956: Policy loss: 2.847620. Value loss: 18.051216. Entropy: 0.118830.
Iteration 3957: Policy loss: 2.492115. Value loss: 13.098247. Entropy: 0.116770.
episode: 1636   score: 130.0  epsilon: 1.0    steps: 536  evaluation reward: 209.45
episode: 1637   score: 235.0  epsilon: 1.0    steps: 788  evaluation reward: 210.75


Iteration 4009: Policy loss: -0.434667. Value loss: 19.148506. Entropy: 0.125453.
Iteration 4010: Policy loss: -0.777641. Value loss: 14.365764. Entropy: 0.112538.
Iteration 4011: Policy loss: -0.660092. Value loss: 11.808970. Entropy: 0.126076.
episode: 1664   score: 165.0  epsilon: 1.0    steps: 470  evaluation reward: 193.0
episode: 1665   score: 105.0  epsilon: 1.0    steps: 599  evaluation reward: 192.25
episode: 1666   score: 65.0  epsilon: 1.0    steps: 1010  evaluation reward: 189.95
Training network. lr: 0.000219. clip: 0.087715
Iteration 4012: Policy loss: 1.600743. Value loss: 17.493425. Entropy: 0.378607.
Iteration 4013: Policy loss: 1.545107. Value loss: 11.380888. Entropy: 0.486691.
Iteration 4014: Policy loss: 1.607961. Value loss: 9.041997. Entropy: 0.476556.
Training network. lr: 0.000219. clip: 0.087715
Iteration 4015: Policy loss: 0.772930. Value loss: 26.121996. Entropy: 0.483861.
Iteration 4016: Policy loss: 0.925399. Value loss: 13.924390. Entropy: 0.488540.
Itera

Training network. lr: 0.000219. clip: 0.087568
Iteration 4069: Policy loss: -0.693861. Value loss: 19.899605. Entropy: 0.327034.
Iteration 4070: Policy loss: -0.633094. Value loss: 12.154847. Entropy: 0.330613.
Iteration 4071: Policy loss: -0.638956. Value loss: 8.152069. Entropy: 0.322681.
episode: 1693   score: 210.0  epsilon: 1.0    steps: 56  evaluation reward: 197.05
Training network. lr: 0.000219. clip: 0.087568
Iteration 4072: Policy loss: 2.242862. Value loss: 25.694790. Entropy: 0.291980.
Iteration 4073: Policy loss: 2.373662. Value loss: 18.971207. Entropy: 0.273437.
Iteration 4074: Policy loss: 2.320723. Value loss: 15.911024. Entropy: 0.293576.
episode: 1694   score: 285.0  epsilon: 1.0    steps: 395  evaluation reward: 196.2
Training network. lr: 0.000219. clip: 0.087568
Iteration 4075: Policy loss: -0.045041. Value loss: 26.658865. Entropy: 0.430292.
Iteration 4076: Policy loss: 0.064225. Value loss: 21.375977. Entropy: 0.445840.
Iteration 4077: Policy loss: -0.022159. Va

In [ ]:
torch.save(agent.policy_net, "./save_model/spaceinvaders_ppo")

In [ ]:
### Loop through all environments and run PPO on them

env_names = ['Atlantis-v0', 'Alien-v0', 'Amidar-v0', 'Assault-v0', 'Asterix-v0', 'Asteroids-v0', 'BankHeist-v0', 'MsPacman-v0']

for a in range(len(env_names)):
    
    name = env_names[a]
    
    envs = []
    for i in range(num_envs):
        envs.append(GameEnv(name))
    #env.render()

    number_lives = envs[0].life
    state_size = envs[0].observation_space.shape
    action_size = envs[0].action_space.n
    rewards, episodes = [], []

    vis_env_idx = 0
    vis_env = envs[vis_env_idx]
    e = 0
    frame = 0
    max_eval = -np.inf
    reset_count = 0


    agent = Agent(action_size)
    torch.save(agent.policy_net.state_dict(), "./save_model/" + name + "_best")
    evaluation_reward = deque(maxlen=evaluation_reward_length)
    frame = 0
    memory_size = 0
    reset_max = 10
    
    print("Determing min/max rewards of environment")
    [low, high] = score_range = get_score_range(name)
    print("Min: %d. Max: %d." % (low, high))

    while (frame < 10000000):
        step = 0
        assert(num_envs * env_mem_size == train_frame)
        frame_next_vals = []
        for i in range(num_envs):
            env = envs[i]
            #history = env.history
            #life = env.life
            #state, reward, done, info = [env.state, env.reward, env.done, env.info]
            for j in range(env_mem_size):
                step += 1
                frame += 1

                curr_state = env.history[HISTORY_SIZE-1,:,:]
                action, value = agent.get_action(np.float32(env.history[:HISTORY_SIZE,:,:]) / 255.)

                next_state, env.reward, env.done, env.info = env.step(action)

                if (i == vis_env_idx):
                    vis_env._env.render()

                frame_next_state = get_frame(next_state)
                env.history[HISTORY_SIZE,:,:] = frame_next_state
                terminal_state = check_live(env.life, env.info['ale.lives'])

                env.life = env.info['ale.lives']
                r = ((env.reward - low) / (high - low)) * 10

                agent.memory.push(i, deepcopy(curr_state), action, r, terminal_state, value, 0, 0)
                if (j == env_mem_size-1):
                    _, frame_next_val = agent.get_action(np.float32(env.history[1:,:,:]) / 255.)
                    frame_next_vals.append(frame_next_val)
                env.score += env.reward
                env.history[:HISTORY_SIZE, :, :] = env.history[1:,:,:]

                if (env.done):
                    if (e % 50 == 0):
                        print('now time : ', datetime.now())
                        rewards.append(np.mean(evaluation_reward))
                        episodes.append(e)
                        pylab.plot(episodes, rewards, 'b')
                        pylab.savefig("./save_graph/" + name + "_ppo.png")
                        torch.save(agent.policy_net, "./save_model/" + name + "_ppo")

                        if np.mean(evaluation_reward) > max_eval:
                            torch.save(agent.policy_net.state_dict(), "./save_model/"  + name + "_ppo_best")
                            max_eval = float(np.mean(evaluation_reward))
                            reset_count = 0
                        elif e > 5000:
                            reset_count += 1
                            """
                            if (reset_count == reset_max):
                                print("Training went nowhere, starting again at best model")
                                agent.policy_net.load_state_dict(torch.load("./save_model/spaceinvaders_ppo_best"))
                                agent.update_target_net()
                                reset_count = 0
                            """
                    e += 1
                    evaluation_reward.append(env.score)
                    print("episode:", e, "  score:", env.score,  " epsilon:", agent.epsilon, "   steps:", step,
                      " evaluation reward:", np.mean(evaluation_reward))

                    env.done = False
                    env.score = 0
                    env.history = np.zeros([HISTORY_SIZE+1,84,84], dtype=np.uint8)
                    env.state = env.reset()
                    env.life = number_lives
                    get_init_state(env.history, env.state)



        agent.train_policy_net(frame, frame_next_vals)
        agent.update_target_net()

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/model.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(x[:,:self.action_size] - torch.max(x[:,:self.action_size],1)[0].unsqueeze(1))


Training network. lr: 0.000250. clip: 0.100000


/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:255: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  pol_loss += pol_avg.detach().cpu()[0]
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:256: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  vf_loss += value_loss.detach().cpu()[0]
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:257: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  ent_total += ent.detach().cpu()[0]


Iteration 1: Policy loss: -0.134242. Value loss: 0.025672. Entropy: 1.384760.
Iteration 2: Policy loss: -0.134622. Value loss: 0.024417. Entropy: 1.382654.
Iteration 3: Policy loss: -0.138663. Value loss: 0.025462. Entropy: 1.380519.
Training network. lr: 0.000250. clip: 0.100000
Iteration 4: Policy loss: -0.166682. Value loss: 0.051994. Entropy: 1.374561.
Iteration 5: Policy loss: -0.153367. Value loss: 0.041330. Entropy: 1.375694.
Iteration 6: Policy loss: -0.167738. Value loss: 0.047337. Entropy: 1.376785.
Training network. lr: 0.000250. clip: 0.100000
Iteration 7: Policy loss: -0.256462. Value loss: 0.727763. Entropy: 1.370962.
Iteration 8: Policy loss: -0.249125. Value loss: 0.479165. Entropy: 1.374830.
Iteration 9: Policy loss: -0.237247. Value loss: 0.508010. Entropy: 1.382150.
Training network. lr: 0.000250. clip: 0.100000
Iteration 10: Policy loss: -0.563716. Value loss: 2.374101. Entropy: 1.376569.
Iteration 11: Policy loss: -0.543814. Value loss: 2.030069. Entropy: 1.379307.

/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Training network. lr: 0.000250. clip: 0.100000
Iteration 28: Policy loss: -0.035380. Value loss: 1.883276. Entropy: 1.367360.
Iteration 29: Policy loss: -0.036834. Value loss: 1.759964. Entropy: 1.370840.
Iteration 30: Policy loss: -0.036598. Value loss: 1.746900. Entropy: 1.371950.
Training network. lr: 0.000250. clip: 0.100000
Iteration 31: Policy loss: -0.058636. Value loss: 1.006419. Entropy: 1.369001.
Iteration 32: Policy loss: -0.033320. Value loss: 0.862457. Entropy: 1.364149.
Iteration 33: Policy loss: -0.065974. Value loss: 0.884775. Entropy: 1.361571.
Training network. lr: 0.000250. clip: 0.100000
Iteration 34: Policy loss: 0.000143. Value loss: 1.397101. Entropy: 1.370030.
Iteration 35: Policy loss: -0.000328. Value loss: 1.279787. Entropy: 1.359337.
Iteration 36: Policy loss: -0.010333. Value loss: 1.356722. Entropy: 1.366242.
Training network. lr: 0.000250. clip: 0.100000
Iteration 37: Policy loss: 0.151668. Value loss: 0.326707. Entropy: 1.362033.
Iteration 38: Policy los

Iteration 103: Policy loss: 0.104650. Value loss: 0.487069. Entropy: 1.310694.
Iteration 104: Policy loss: 0.121857. Value loss: 0.390418. Entropy: 1.308963.
Iteration 105: Policy loss: 0.107936. Value loss: 0.398923. Entropy: 1.303240.
Training network. lr: 0.000249. clip: 0.099696
Iteration 106: Policy loss: -0.019600. Value loss: 0.810460. Entropy: 1.291222.
Iteration 107: Policy loss: -0.043799. Value loss: 0.813314. Entropy: 1.305367.
Iteration 108: Policy loss: -0.041227. Value loss: 0.788016. Entropy: 1.295246.
Training network. lr: 0.000249. clip: 0.099696
Iteration 109: Policy loss: -0.053537. Value loss: 1.527595. Entropy: 1.300061.
Iteration 110: Policy loss: -0.172127. Value loss: 1.685091. Entropy: 1.297742.
Iteration 111: Policy loss: -0.111701. Value loss: 1.285365. Entropy: 1.288221.
Training network. lr: 0.000249. clip: 0.099696
Iteration 112: Policy loss: 0.182973. Value loss: 0.846592. Entropy: 1.276863.
Iteration 113: Policy loss: 0.123608. Value loss: 0.738449. Ent

episode: 27   score: 15100.0  epsilon: 1.0    steps: 652  evaluation reward: 20807.40740740741
Training network. lr: 0.000249. clip: 0.099548
Iteration 178: Policy loss: 0.024522. Value loss: 1.066570. Entropy: 1.251285.
Iteration 179: Policy loss: 0.002568. Value loss: 0.774038. Entropy: 1.244492.
Iteration 180: Policy loss: 0.035311. Value loss: 0.542031. Entropy: 1.251379.
Training network. lr: 0.000249. clip: 0.099548
Iteration 181: Policy loss: 0.348849. Value loss: 0.860963. Entropy: 1.250753.
Iteration 182: Policy loss: 0.360043. Value loss: 0.543085. Entropy: 1.267107.
Iteration 183: Policy loss: 0.340807. Value loss: 0.445778. Entropy: 1.261358.
Training network. lr: 0.000249. clip: 0.099548
Iteration 184: Policy loss: -0.182871. Value loss: 1.296346. Entropy: 1.253929.
Iteration 185: Policy loss: -0.199198. Value loss: 1.305865. Entropy: 1.246943.
Iteration 186: Policy loss: -0.209038. Value loss: 1.193934. Entropy: 1.243795.
Training network. lr: 0.000249. clip: 0.099548
Ite